<a href="https://colab.research.google.com/github/zarifi/3311/blob/master/Copy_of_assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np
import scipy.stats
import copy
from sklearn.model_selection import KFold

################# entropoy
# Tip: feel free to call scipy.stats.entropy.  Make sure to use log base 2.
#
# Input:
# data_frame -- pandas data frame
#
# Output:
# answer -- float indicating the empirical entropy of tyhe data in data_frame
##############################################
def entropy(data_frame):

	# default value until this function is filled in
	answer = 0
	high_count = data_frame.loc[data_frame['class'] == 'high'].shape[0]
	low_count = data_frame.loc[data_frame['class'] == 'low'].shape[0]
	# print("high_count: ",high_count)
	total = high_count + low_count
	# print("total: ", total)
	if total == 0:
		return 0
	pk = np.array([low_count/total, high_count/total])
	answer = scipy.stats.entropy(pk,base=2)
	return answer

################# info_gain
# Tip: this function should call entropy
#
# Inputs:
# data_frame -- pandas data frame
# attribute -- string indicating the attribute for which we wish to compute the information gain
# domain -- set of values (strings) that the attribute can take
#
# Output:
# answer -- float indicating the information gain
################################################3
def info_gain(data_frame, attribute, domain):

	# default value until this function is filled in
	answer = 0
	remainder_attribute = 0
	for d in domain:
		data_frame_subset = data_frame.loc[data_frame[attribute] == d]
		weight = data_frame_subset.shape[0]/data_frame.shape[0]
		# print("weight: ", weight)
		# print("entropy(data_frame_subset): ",entropy(data_frame_subset))
		remainder_attribute += weight * entropy(data_frame_subset)
	# print("entropy(data_frame): ", entropy(data_frame))
	answer = entropy(data_frame) - remainder_attribute
	return answer

######## Decision_tree class
#
# This class defines the data structure of the decision tree to be learnt
############################
class Decision_Tree:

	# constructor
	def __init__(self,attribute,branches,label):
		self.attribute = attribute
		self.branches = branches
		self.label = label

	# leaf constructor
	def make_leaf(label):
		return Decision_Tree('class', {}, label)

	# node constructor
	def make_node(attribute,branches):
		return Decision_Tree(attribute, branches, None)

	# string representation
	def __repr__(self):
		return self.string_repr(0)

	# decision tree string representation
	def string_repr(self,indent):
		indentation = '\t'*indent

		# leaf string representation
		if self.attribute == 'class':
			return f'\n{indentation}class = {self.label}'

		# node string representation
		else:
			representation = ''
			for value in self.branches:
				representation += f'\n{indentation}{self.attribute} = {value}:'
				representation += self.branches[value].string_repr(indent+1)
			return representation

	# classify a data point
	def classify(self,data_point):

		# leaf
		if self.attribute == 'class':
			return self.label

		# node
		else:
			return self.branches[data_point[self.attribute]].classify(data_point)

############# choose attribute
# Tip: this function should call info_gain
#
# Inputs:
# attributes_with_domains -- dictionary with attributes as keys and domains as values
# data_frame -- pandas data_frame
#
# Output:
# best_score -- float indicting the information gain score of the best attribute
# best_attribute -- string indicating the best attribute
#################################
def choose_attribute(attributes_with_domains,data_frame):

	# default value until this function is filled in
	best_score = 0
	best_attribute = 'NA'
	for attribute in attributes_with_domains:
		domain = attributes_with_domains[attribute]
		information_gain = info_gain(data_frame,attribute,domain)
		# print("information_gain: ", information_gain)
		if information_gain > best_score:
			best_score = information_gain
			best_attribute = attribute
	return best_score, best_attribute

############# train decision tree
# Tip: this is a recursive function that should call itself as well as
# choose_attribute,  Decision_Tree.make_leaf, Decision_Tree.make_node
#
# Inputs:
# data_frame -- pandas data frame
# attributes_with_domains -- dictionary with attributes as keys and domains as values
# default_class -- string indicating the class to be assigned when data_frame is empty
# threshold -- integer indicating the minimum number of data points in data_frame to allow
#              the creation of a new node that splits the data with some attribute
#
# Output:
# decision_tree -- Decision_Tree object
########################
def train_decision_tree(data_frame,attributes_with_domains,default_class,threshold):

	# default value until this function is filled in
	if data_frame.shape[0] < threshold:
		return Decision_Tree.make_leaf(default_class)
	elif (data_frame['class'] == 'high').all() or (data_frame['class'] == 'low').all():
		# print("label: ",data_frame['class'].iloc[0])
		label = data_frame['class'].iloc[0]
		return Decision_Tree.make_leaf(label)
	elif not attributes_with_domains:
		high_count = data_frame.loc[data_frame['class'] == 'high'].shape[0]
		low_count = data_frame.loc[data_frame['class'] == 'low'].shape[0]
		if high_count > low_count:
			return Decision_Tree.make_leaf('high')
		else:
			return Decision_Tree.make_leaf('low')

	best_score, best_attribute = choose_attribute(attributes_with_domains,data_frame)
	# print(best_attribute)
	domains = attributes_with_domains[best_attribute]
	branches = {}
	for value in domains:
		examples_i = data_frame.loc[data_frame[best_attribute] == value]
		updated_attributes = attributes_with_domains.copy()
		del updated_attributes[best_attribute]
		high_count = examples_i.loc[examples_i['class'] == 'high'].shape[0]
		low_count = examples_i.loc[examples_i['class'] == 'low'].shape[0]
		class_mode = 'low'
		if high_count > low_count:
			class_mode = 'high'
		subtree = train_decision_tree(examples_i,updated_attributes,class_mode,threshold)
		# print("subtree: ",subtree)
		branches[value] = subtree
		# tree.branches[value] = subtree
		# tree = Decision_Tree.make_node(value,subtree)
	tree = Decision_Tree.make_node(best_attribute,branches)

	return tree

	# return Decision_Tree.make_leaf(default_class)

######### eval decision tree
# Tip: this function should call decision_tree.classify
#
# Inputs:
# decision tree -- Decision_Tree object
# data_frame -- pandas data frame
#
# Output:
# accuracy -- float indicating the accuracy of the decision tree
#############
def eval_decision_tree(decision_tree, data_frame):

	# default value until this function is filled in
	accuracy = 0
	total = 0
	correct = 0
	for index, row in data_frame.iterrows():
		true_label = row['class']
		# print(index, dict(row))
		predicted_label = decision_tree.classify(dict(row))
		# print(predicted_label)
		total += 1
		if predicted_label == true_label:
			correct += 1
	accuracy = correct/total
	# print("correct: ", correct)
	# print("total: ", total)
	return accuracy

########### k-fold cross-validation
# Tip: this function should call train_decision_tree and eval_decision_tree
#
# Inputs:
# train_data -- pandas data frame
# test_data -- pandas data frame
# attributes_with_domains -- dictionary with attributes as keys and domains as values
# k -- integer indicating the number of folds
# threshold_list -- list of thresholds to be evaluated
#
# Outputs:
# best_threshold -- integer indiating the best threshold found by cross validation
# test_accuracy -- float indicating the accuracy based on the test set
#####################################3
def cross_validation(train_data, test_data, attributes_with_domains, k, threshold_list):

	# default value until this function is filled in
	best_threshold = threshold_list[0]
	test_accuracy = 0
	best_validation_accuracy = 0
	# data_x = train_data.drop('class', axis=1)
	# data_y = train_data['class']
	# print(train_data_y)
	kf = KFold(n_splits=k)
	for threshold in threshold_list:
		for train_index, validation_index in kf.split(train_data):
			train_data_split = train_data.iloc[train_index]
			validation_data_split = train_data.iloc[validation_index]
			# train_data_y = data_y.iloc[train_index]
			# validation_data_y = data_y.iloc[validation_index]
			trained_decision_tree = train_decision_tree(train_data_split,attributes_with_domains,'low',threshold)
			validation_accuracy = eval_decision_tree(trained_decision_tree,validation_data_split)
			if validation_accuracy > best_validation_accuracy:
				best_validation_accuracy = validation_accuracy
				best_threshold = threshold
	test_accuracy = eval_decision_tree(trained_decision_tree, test_data)
	return best_threshold, test_accuracy

############################ main
# You should not need to change the code below
#
# This code performs the following operations:
# 1) Load the data
# 2) create a list of attributes
# 3) create a dictionary that maps each attribute to its domain of values
# 4) split the data into train and test sets
# 5) train a decision tree while optimizing the threshold hyperparameter by
#    10-fold cross validation
#####################################

#load data
data_frame = pd.read_csv("categorical_real_estate.csv")
data_frame = data_frame.fillna('NA')
# print(data_frame)

# get attributes
attributes = list(data_frame.columns)
attributes.remove('class')

# create dictionary that maps each attribute to its domain of values
attributes_with_domains = {}
for attr in attributes:
	attributes_with_domains[attr] = set(data_frame[attr])

print(attributes_with_domains)
#split data in to train and test
train_data = data_frame.iloc[0:1000]
test_data = data_frame.iloc[1000:]
# t_data = train_data.iloc[0:3]
# print(t_data['class'][0])
# perform 10-fold cross-validation
best_threshold, accuracy = cross_validation(train_data, test_data, attributes_with_domains, 10, [10,20,40,80,160])
print(f'Best threshold {best_threshold}: accuracy {accuracy}')



{'MSZoning': {'RL', 'C (all)', 'FV', 'RM', 'RH'}, 'Street': {'Grvl', 'Pave'}, 'Alley': {'Grvl', 'Pave', 'NA'}, 'LotShape': {'IR1', 'IR2', 'IR3', 'Reg'}, 'LandContour': {'Lvl', 'Low', 'HLS', 'Bnk'}, 'Utilities': {'NoSeWa', 'AllPub'}, 'LotConfig': {'FR2', 'CulDSac', 'FR3', 'Inside', 'Corner'}, 'LandSlope': {'Mod', 'Sev', 'Gtl'}, 'Neighborhood': {'Edwards', 'StoneBr', 'Somerst', 'Gilbert', 'Sawyer', 'NoRidge', 'NWAmes', 'NridgHt', 'CollgCr', 'Blueste', 'BrkSide', 'Mitchel', 'NPkVill', 'SWISU', 'Timber', 'OldTown', 'SawyerW', 'IDOTRR', 'MeadowV', 'Crawfor', 'ClearCr', 'NAmes', 'Blmngtn', 'Veenker', 'BrDale'}, 'Condition1': {'Artery', 'Feedr', 'RRNe', 'RRNn', 'PosA', 'PosN', 'Norm', 'RRAn', 'RRAe'}, 'Condition2': {'Artery', 'Feedr', 'RRNn', 'PosA', 'PosN', 'Norm', 'RRAn', 'RRAe'}, 'BldgType': {'2fmCon', '1Fam', 'Duplex', 'Twnhs', 'TwnhsE'}, 'HouseStyle': {'2Story', '2.5Unf', 'SFoyer', '1.5Unf', 'SLvl', '1.5Fin', '1Story', '2.5Fin'}, 'RoofStyle': {'Hip', 'Gable', 'Flat', 'Gambrel', 'Shed', '